# Lab 01
* João Victor E. Matoso
* 176293

# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [1]:
#include <stdio.h>

int main(){
    // Entradas do problema:
    double s = 200;
    double j = 1;
    int n = 5;
    
    // Calculo iterativo das parcelas até n:
    for(int i = 1; i <= n; i++){
        printf("Parcela %d: %.2f\n", i, s);
        s += s*j/100;
    }
    
    return 0;
}

Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [2]:
#include <stdio.h>

double proximaParcela(int x, double s, double j, int n){
    // Verifica se parcela faz parte do emprestimo
    if(x>n)
        return 0;
    
    // Calculo da parcela s = s*(1+j/100)^(x-1)
    for(int i = 0; i < x-1; i++)
        s *= (1+j/100);
    
    return s;
}

int main(){
    // Entradas do problema:
    double s = 200;
    double j = 1;
    int n = 5;
    
    // Calculo de 7 parcelas do emprestimo:
    for(int i = 1; i <= 7; i++)
        printf("Parcela %d: %.2f\n", i, proximaParcela(i, s, j, n));
    
    return 0;
}


Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12
Parcela 6: 0.00
Parcela 7: 0.00


## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [3]:
#include <stdio.h>

// Variaveis globais de estado do emprestimo:
double s;
double j;
int n;
int parcelaAtual;

// Função de inicialização do emprestimo
// Atualiza as variaveis globais de estado do emprestimo
void inicializaEmprestimo(double primeiraParcela, double juros, int totalParcelas){
    s = primeiraParcela;
    j = juros;
    n = totalParcelas;
    parcelaAtual = 1;
}


// Função para o calculo da proxima parcela do emprestimo
double proximaParcela(){
    double retorno = s;
    
    // Verifica se emprestimo não chegou ao fim
    if(parcelaAtual < n)
        s += s*j/100;
    else
        s = 0;
    
    parcelaAtual++;
    
    return retorno;
}


int main(){
    // Inicialização do emprestimo
    inicializaEmprestimo(200,1,5);
    
    // Calculo de 7 parcelas do emprestimo
    for(int i = 1; i <= 7; i++)
        printf("Parcela %d: %.2f\n", i, proximaParcela());
    
    return 0;
}


Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12
Parcela 6: 0.00
Parcela 7: 0.00


## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [4]:
#include <stdio.h>


// Estrutura para guardar variaveis de estado do emprestimo
typedef struct {
    double parcelaAtual;
    double juros;
    int n;
    int i;
} Emprestimo;


// Função para o calculo da proxima parcela do emprestimo
double proximaParcela(Emprestimo *e){
    double s = e->parcelaAtual;
    
    // Verifica se emprestimo não chegou ao fim
    if(e->i < e->n)
        e->parcelaAtual *= (1+e->juros/100);
    else
        e->parcelaAtual = 0;
    
    (e->i)++;
    
    return s;
}


int main(){
    // Entradas do problema
    Emprestimo e = {200, 1, 5, 1};
        
    // Calculo de 7 parcelas do emprestimo
    for(int i = 1; i <= 7; i++)
        printf("Parcela %d: %.2f\n", i, proximaParcela(&e));
    
    return 0;
}

Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12
Parcela 6: 0.00
Parcela 7: 0.00


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [5]:
#include <stdio.h>

typedef struct {
    double parcelaAtual;
    double juros;
    int n;
    int i;
} Emprestimo;


// Função para o calculo da proxima parcela do emprestimo
double proximaParcela(Emprestimo *e){
    double s = e->parcelaAtual;
    
    // Verifica se emprestimo não chegou ao fim
    if(e->i < e->n)
        e->parcelaAtual *= (1+e->juros/100);
    else
        e->parcelaAtual = 0;
    
    (e->i)++;
    
    return s;
}


int main(){
    // Entradas do problema:
    Emprestimo e1 = {200, 1, 5, 1};
    Emprestimo e2 = {500, 2, 7, 1};
        
    // Calculo de 7 parcelas de cada emprestimo:
    double parcela;
    
    for(int i = 1; i <= 7; i++){
        parcela = proximaParcela(&e1);
        
        // Evita a impressão de parcelas iguais a zero
        if(parcela)
            printf("Emprestimo 1: parcela %d eh %.2f\n", i, parcela);
        
        parcela = proximaParcela(&e2);
        
        // Evita a impressão de parcelas iguais a zero
        if(parcela)
            printf("Emprestimo 2: parcela %d eh %.2f\n", i, parcela);
    }
    
    return 0;
}

Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
